In [1]:
!pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import csv
import json
import time
import random
import requests
import pandas as pd
import zipfile
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from google.colab import files

# ========== CONFIGURACIÓN ========== #
CARPETA_PORTADAS = "portadas"
CSV_FILE = "libros.csv"
LETRA = 'W'

if not os.path.exists(CARPETA_PORTADAS):
    os.makedirs(CARPETA_PORTADAS)

# ========== FUNCIONES ========== #

def guardar_portada(url_imagen, titulo, autor_principal):
    nombre_archivo = f"{titulo} - {autor_principal}"
    nombre_archivo = re.sub(r'[\\/*?:"<>|]', "", nombre_archivo) + ".jpg"
    ruta = os.path.join(CARPETA_PORTADAS, nombre_archivo)
    try:
        respuesta = requests.get(url_imagen, timeout=15)
        if respuesta.status_code == 200:
            with open(ruta, "wb") as f:
                f.write(respuesta.content)
            return ruta
    except Exception as e:
        print(f"❌ Error al descargar imagen: {e}")
    return ""

def guardar_en_csv(fila):
    archivo_existe = os.path.exists(CSV_FILE)
    with open(CSV_FILE, "a", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        if not archivo_existe:
            encabezados = ["Título", "Autor/es", "Género/s", "Descripción", "Ruta portada"]
            writer.writerow(encabezados)
        writer.writerow(fila)

def procesar_libro(driver, url_libro):
    try:
        driver.get(url_libro)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        titulo_tag = soup.select_one("div#title h1")
        titulo = titulo_tag.text.strip() if titulo_tag else "Desconocido"

        autores = []
        bloque_autores = soup.find("div", id="autor")
        if bloque_autores:
            autores = [a.text.strip() for a in bloque_autores.find_all("a")]
        autor_principal = autores[0] if autores else "Desconocido"

        generos = []
        bloque_generos = soup.find("div", id="genero")
        if bloque_generos:
            generos = [a.text.strip() for a in bloque_generos.find_all("a")]

        descripcion = ""
        bloque_desc = soup.find("div", id="sinopsis")
        if bloque_desc:
            texto = bloque_desc.find("div", class_="ali_justi")
            if texto:
                descripcion = texto.get_text(separator=" ").strip()
            else:
                descripcion = bloque_desc.get_text(separator=" ").strip()

        ruta_imagen = ""
        img_tag = soup.select_one("div#cover img")
        if img_tag and img_tag.get("src"):
            url_imagen = img_tag["src"]
            ruta_imagen = guardar_portada(url_imagen, titulo, autor_principal)

        fila = [titulo, ", ".join(autores), ", ".join(generos), descripcion, ruta_imagen]
        guardar_en_csv(fila)
        print(f"✅ Guardado: {titulo}")

    except Exception as e:
        print(f"⚠️ Error procesando libro {url_libro}: {e}")

# ========== INICIO ========== #

import google_colab_selenium as gs

driver = gs.Chrome()
driver.get("https://ww3.lectulandia.com/?loggedout=true")
time.sleep(3)

WebDriverWait(driver, 10).until(lambda d: d.execute_script("return typeof jQuery !== 'undefined';"))

print(f"\n🔤 Procesando letra: {LETRA}")

driver.execute_script(f"""
    window.lectuAutores = null;
    jQuery.post('https://ww3.lectulandia.com/wp-admin/admin-ajax.php', {{
        action: 'PDOTaxonomies',
        letra: '{LETRA}',
        term: 'autor'
    }}, function(data) {{
        window.lectuAutores = data;
    }});
""")

try:
    WebDriverWait(driver, 10).until(lambda d: d.execute_script("return window.lectuAutores !== null"))
    autores_raw = driver.execute_script("return window.lectuAutores;")
    autores_dict = json.loads(autores_raw) if isinstance(autores_raw, str) else autores_raw
    lista_autores = list(autores_dict.values())

except Exception as e:
    print(f"⚠️ No se pudieron obtener autores para letra {LETRA}: {e}")
    driver.quit()
    raise

for nombre_autor in lista_autores:
    url_autor = f"https://ww3.lectulandia.com/autor/{nombre_autor}"
    print(f"✍️ Autor: {nombre_autor}")
    try:
        driver.get(url_autor)
        #time.sleep(2)
        soup_autor = BeautifulSoup(driver.page_source, "html.parser")
        libros = soup_autor.find_all("article", class_="card")

        for libro in libros:
            titulo = libro.find("img", class_="cover")["alt"].strip()
            enlace = libro.find("a", class_="title")["href"].strip()
            url_libro = f"https://ww3.lectulandia.com{enlace}"
            print(f"📚 Libro: {titulo}")
            procesar_libro(driver, url_libro)
            #time.sleep(random.uniform(1.5, 3.0))

    except Exception as e:
        print(f"⚠️ Error con autor {nombre_autor}: {e}")

driver.quit()
print("\n🎉 Scraping finalizado.")

# ========== DESCARGA DE RESULTADOS ========== #

# 📄 Mostrar el CSV
df = pd.read_csv(CSV_FILE)
df.head()

# 💾 Descargar CSV
files.download(CSV_FILE)

# 📦 Comprimir carpeta de portadas
zip_file = "portadas.zip"
with zipfile.ZipFile(zip_file, 'w') as zipf:
    for root, _, files_in_dir in os.walk(CARPETA_PORTADAS):
        for file in files_in_dir:
            zipf.write(os.path.join(root, file))

# ⬇️ Descargar ZIP
files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


🔤 Procesando letra: W
✍️ Autor: w-a-harbinson/
📚 Libro: Génesis
✅ Guardado: Génesis
✍️ Autor: w-ama/
📚 Libro: Haz que suceda
✅ Guardado: Haz que suceda
📚 Libro: Vuela alto
✅ Guardado: Vuela alto
📚 Libro: Para quien se atreve a soñar
✅ Guardado: Para quien se atreve a soñar
📚 Libro: Donde todo es posible
✅ Guardado: Donde todo es posible
📚 Libro: Siempre hay un lugar
✅ Guardado: Siempre hay un lugar
📚 Libro: Descubre el camino
✅ Guardado: Descubre el camino
📚 Libro: Pide un deseo
✅ Guardado: Pide un deseo
📚 Libro: Brilla entre las nubes
✅ Guardado: Brilla entre las nubes
📚 Libro: Cuenta conmigo
✅ Guardado: Cuenta conmigo
📚 Libro: Sigue adelante
✅ Guardado: Sigue adelante
📚 Libro: Elige tu estrella
✅ Guardado: Elige tu estrella
📚 Libro: Todas para una
✅ Guardado: Todas para una
📚 Libro: Lo mejor de cada una
✅ Guardado: Lo mejor de cada una
✍️ Autor: w-b-bartlett/
📚 Libro: Los asesinos
✅ Guardado: Los asesinos
✍️ Autor: w-bernard-carlson/
📚 Libro: Tesla: inventor de la era eléctrica
✅ Gu